In [6]:
import numpy as np
import mlflow.azureml
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.43.0


## deploying the model on Azure Cotainer Instances (ACI)

### initialize workspace

In [7]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, sep = '\n')

MLOps_WS
Learn_MLOPs
northeurope


In [8]:
scaler = Model(ws, 'scaler').download(exist_ok=True)

In [9]:
model = Model(ws, 'support-vector-classifier').download(exist_ok=True)

In [10]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

UserErrorException: UserErrorException:
	Message: azureml.mlflow could not be imported. Please ensure that 'azureml-mlflow' has been installed in the current python environment.
	InnerException No module named 'azureml.mlflow'
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "azureml.mlflow could not be imported. Please ensure that 'azureml-mlflow' has been installed in the current python environment."
    }
}

## create a scoring script

In [ ]:
%%writefile score.py
import json
import os
import pickle
import joblib
import onnxruntime
import time
import numpy as np
from azureml.core.model import Model

In [ ]:
def init():
    global model, scaler, input_name, label_name
    scaler_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'scaler/1/scaler.pkl')
    # deserialize the model file back into a sklearn model
    scaler = joblib.load(scaler_path)

    model_onnx = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'support-vector-classifier/1/svc.onnx')
    # deserialize the svc model
    model = onnxruntime.InferenceSession(model_onnx, None)
    input_name = model.get_inputs()[0].name
    label_name = model.get_outputs()[0].name


def run(raw_data):
    try:
        data = np.array(json.loads(raw_data)['data']).astype('float32')
        data = scaler.fit_transform(data.reshape(1, 7))

        #make predictions
        model_prediction = model.run([label_name], {input_name: data.astype(np.float32)})[0]
    except Exception as e:
        model_prediction = 'error'
    
    return model_prediction